# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [71]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [72]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [73]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'fruitsDB', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [74]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [75]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [76]:
# review the collections in our new database


In [77]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('65a89f1aa049aa900a8426b4'), 'FHRSID': 254250, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000066174', 'BusinessName': 'The Pines Calyx', 'BusinessType': 'Other catering premises', 'BusinessTypeID': 7841, 'AddressLine1': 'The Pines Garden', 'AddressLine2': 'Beach Road', 'AddressLine3': 'St Margarets Bay', 'AddressLine4': 'Kent', 'PostCode': 'CT15 6DZ', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2021-08-17T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'}, 'RightToReply': '', 'Distance': 4587.362402580997, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'to

In [78]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [79]:
# Create a dictionary for the new restaurant data
restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [80]:
# Insert the new restaurant into the collection
establishments.insert_one(restaurant)

In [81]:
# Check that the new restaurant was inserted
print(establishments)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [83]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = db.establishments.find(
  { "BusinessType": "Restaurant/Cafe/Canteen" },
  { "_id": 0, "BusinessTypeID": 1, "BusinessType": 1 }
);

result = query.next()
result

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [88]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName": "Penang Flavours"}
update = {"$set": {"BusinessTypeID": 1}}
result = establishments.update_one(query, update)

In [89]:
# Confirm that the new restaurant was updated
if result.modified_count > 0:
    print("Document updated successfully")
else:
    print("No documents matched the query or no changes were made")

Document updated successfully


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [92]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [94]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
result = establishments.delete_many(query)

No documents matched the query or no documents were deleted


In [95]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 0


In [96]:
# Check that other documents remain with 'find_one'
query = {"LocalAuthorityName": {"$ne": "Dover"}}
remaining_document = establishments.find_one(query)

if remaining_document:
    print("There are remaining documents after deletion:")
    print(remaining_document)
else:
    print("No remaining documents after deletion")

There are remaining documents after deletion:
{'_id': ObjectId('65a89f1aa049aa900a84299b'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': '4', 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [102]:
# Change the data type from String to Decimal for longitude and latitude
update = [
    {
        '$set': {
            "geocode.longitude": {'$toDouble': "$geocode.longitude"},
            "geocode.latitude": {'$toDouble': "$geocode.latitude"}
        }
    }
]

result = establishments.update_many({}, update)

Use `update_many` to convert `RatingValue` to integer numbers.

In [103]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [105]:
# Change the data type from String to Integer for RatingValue
update = [
    {
        '$set': {
            "RatingValue": {'$toInt': "$RatingValue"}
        }
    }
]

result = establishments.update_many({}, update)

In [107]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()

print(f"Data type of longitude: {type(sample_document['geocode']['longitude'])}")
print(f"Data type of latitude: {type(sample_document['geocode']['latitude'])}")
print(f"Data type of RatingValue: {type(sample_document['RatingValue'])}")


Data type of longitude: <class 'float'>
Data type of latitude: <class 'float'>
Data type of RatingValue: <class 'int'>
